In [1]:
import tensorflow as tf

# attention

attention 最早是应用于计算机视觉，随后在NLP领域中应用。随着
GPT和BERT的大火，attention也随之得到了关注。
attention的核心就是从关注全部到关注重点。

attention两个个优点
1 参数少
2 速度快

早期attention引入nlp中，是和rnn相结合，应用于Machine translation中。对于NMT任务，一般使用seq2seq模型，或者说是encoder-decoder模型。其流程是讲输入语句encoder一个向量，然后通过decoder进行解码，最后输出目标语句。

这种方法存在一些问题，RNN中存在长程梯度消失的问题，对于较长的句子，我们很难寄希望于将输入的序列化为定长的向量而保存所有的有效信息，所以随着所需翻译句子的长度增加，这种结构的效果会显著下降。

attenion 的结构如下

![title](img/attention_v1.png)

attention + RNN做NMT的流程如下：
1）利用RNN结构得到encoder的hidden state(h1, h2, ...hT)
2）假设当前的decoder的hidden state是s_t-1, 我们可以计算每一个输入位置j与当前输出位置的关联性，$e_{ij}=a(s_{t-1}, h_j)$,写成相应的向量形式即为
$\overrightarrow{e_t}=(a(s_{t-1}, h_1),...,a(s_{t-1}, h_T))$, 其$a$是一种相关性的算符，例如常见的有点乘形式$\overrightarrow{e_t}=\overrightarrow{s_{t-1}}\overrightarrow{h}$, 加权点乘$\overrightarrow{e_t}=\overrightarrow{s_{t-1}}W\overrightarrow{h}$, 加和$\overrightarrow{e_t}=\overrightarrow{v}tanh(W_1\overrightarrow{h}+W_2\overrightarrow{s_{t-1}})$

对于$\overrightarrow{t}$进行softmax操作将其normalize得到attention的分布，$\overrightarrow{\alpha_t}=softmax(\overrightarrow{e_t})$, 展开形式为$\alpha_{tj}=\frac{exp(e_{tj})}{\sum_{k=1}^{T}{exp(e_{tk})}}$

利用$\overrightarrow{\alpha_t}$我们可以进行加权求和得到相应的context vector $\overrightarrow{c_t}=\sum_{j=1}^{T}{\alpha_{tj}h_j}$

由此，我们可以计算decoder的下一个hidden state $s_t=f(s_{t-1}, y_{t-1}, c_t)$以及该位置的输入$p(y_t|y_1,...,y_{t-1},\overrightarrow{x})=g(y_{i-1}, s_i, c_i)$

这里的关键操作是计算encoder 和 decoder state 之间的关联的权重，得到attention 分布，从而对于当前输出位置得到比较重要的输入位置的权重，在预测输出时相应的会占较大的比重。